# Preprocessing

Run this notebook before running any analysis scripts.

In [1]:
import sys
sys.path

['/mnt/home/yucel/.conda/envs/as/lib/python36.zip',
 '/mnt/home/yucel/.conda/envs/as/lib/python3.6',
 '/mnt/home/yucel/.conda/envs/as/lib/python3.6/lib-dynload',
 '',
 '/mnt/home/yucel/.conda/envs/as/lib/python3.6/site-packages',
 '/mnt/home/yucel/.conda/envs/as/lib/python3.6/site-packages/argus_shapes-0.2-py3.6-linux-x86_64.egg',
 '/mnt/home/yucel/.conda/envs/as/lib/python3.6/site-packages/argus_scripts-0.1.dev0-py3.6.egg',
 '/mnt/home/yucel/Documents/python-psignifit',
 '/mnt/home/yucel/.conda/envs/as/lib/python3.6/site-packages/IPython/extensions',
 '/mnt/home/yucel/.ipython']

In [2]:
sys.path.append('..')

In [3]:
import os
import six
import numpy as np
import pandas as pd
import skimage as sk
import skimage.io as skio
import os.path as op
import glob
import scipy.ndimage as ndi
from pathlib import Path
import argus_shapes
import argus_scripts
import logging

2021-11-15 19:18:20,609 [pulse2percept] [INFO] Welcome to pulse2percept


In [4]:
#disable Jupyter Notebook handlers
logging.getLogger().handlers = []
logger = logging.getLogger(__name__)
logger.addHandler(logging.StreamHandler())

# Set up root logger for debug file
formatstr = '%(asctime)s  %(message)s'
logging.basicConfig(level=logging.INFO,
                    format=formatstr,
                    filename='.img_selection.log',
                    filemode='w')

In [5]:
datapath = os.environ['VISCOG_DATA']
scriptpath = os.environ['SCRIPT_DATA']

In [6]:
subjects_sessions = [
    ('12-005', ['2019_05_29', '2019_10_09']),
    ('12-104', ['2019_05_30',  '2019_10_08']),
    ('13-101', ['2019_05_31', '2019_10_07'])
]

In [7]:
run_ini2csv = False
run_trials2drawings = False
run_clean2filled = False
run_collatedata = True
run_addimgcol = False
change23 = True

## 1. Convert all .ini files to .csv files

In [8]:
if run_ini2csv:
    for subject, sessions in subjects_sessions:
        for session in sessions:
            !matlab -nosplash -nodesktop -r "all_ini2csv('{datapath}/{subject}/{session}');exit"

## 2. Link trials from csv files with drawings

In [9]:
drawing_folder = {
    'Drawings (single)': [
                        os.path.join('drawings_filled', 'drawings_single')
                        ],
    'Drawings (single-Thr)': [
                        os.path.join('drawings_filled', 'drawings_single')
                        ],
    'Drawings (pairs)': [
                          os.path.join('drawings_filled', 'drawings_double')
                       ],
    'Drawings (pairs - 20Hz)': [
                         os.path.join('drawings_filled', 'drawings_double_20Hz')                          
                          ],
#    'Drawings (tactile)': [os.path.join('drawings_raw', 'drawings_tactile'),
#                           os.path.join('drawings_clean', 'drawings_tactile'),
##                           os.path.join('drawings_filled', 'drawings_tactile')
 #                         ],
  
    'Drawings (pairs-Thr-1)': [
                    os.path.join('drawings_filled', 'drawings_double')
                    ],
    'Drawings (pairs-2Thr-0)': [
                os.path.join('drawings_filled', 'drawings_double')
                ],

    'Drawings (pairs-Thr-0)': [
                os.path.join('drawings_filled', 'drawings_double')
                ],
    'Drawings (pairs-05Thr-0)':[
                os.path.join('drawings_filled', 'drawings_double')
                ],
    'Drawings (pairs-05Thr-1)':[
            os.path.join('drawings_filled', 'drawings_double')
            ]
}    

In [ ]:
# drawing_folder = {
#     'Drawings (single)': [os.path.join('drawings_raw', 'drawings_single'),
#                           os.path.join('drawings_clean', 'drawings_single'),
#                         os.path.join('drawings_filled', 'drawings_single')
#                         ],
#     'Drawings (single-Thr)': [os.path.join('drawings_raw', 'drawings_single'),
#                           os.path.join('drawings_clean', 'drawings_single'),
#                         os.path.join('drawings_filled', 'drawings_single')
#                         ],
#     'Drawings (pairs)': [os.path.join('drawings_raw', 'drawings_double'),
#                     os.path.join('drawings_clean', 'drawings_double'),
#                           os.path.join('drawings_filled', 'drawings_double')
#                        ],
#     'Drawings (pairs - 20Hz)': [os.path.join('drawings_raw', 'drawings_double_20Hz'),
#                          os.path.join('drawings_clean', 'drawings_double_20Hz'),
#                          os.path.join('drawings_filled', 'drawings_double_20Hz')                          
#                           ],
# #    'Drawings (tactile)': [os.path.join('drawings_raw', 'drawings_tactile'),
# #                           os.path.join('drawings_clean', 'drawings_tactile'),
# ##                           os.path.join('drawings_filled', 'drawings_tactile')
#  #                         ],
  
#     'Drawings (pairs-Thr-1)': [os.path.join('drawings_raw', 'drawings_double'),
#                       os.path.join('drawings_clean', 'drawings_double'),
#                     os.path.join('drawings_filled', 'drawings_double')
#                     ],
#     'Drawings (pairs-2Thr-0)': [os.path.join('drawings_raw', 'drawings_double'),
#                   os.path.join('drawings_clean', 'drawings_double'),
#                 os.path.join('drawings_filled', 'drawings_double')
#                 ],

#     'Drawings (pairs-Thr-0)': [os.path.join('drawings_raw', 'drawings_double'),
#                   os.path.join('drawings_clean', 'drawings_double'),
#                 os.path.join('drawings_filled', 'drawings_double')
#                 ],
#     'Drawings (pairs-05Thr-0)':[os.path.join('drawings_raw', 'drawings_double'),
#                   os.path.join('drawings_clean', 'drawings_double'),
#                 os.path.join('drawings_filled', 'drawings_double')
#                 ],
#     'Drawings (pairs-05Thr-1)':[os.path.join('drawings_raw', 'drawings_double'),
#               os.path.join('drawings_clean', 'drawings_double'),
#             os.path.join('drawings_filled', 'drawings_double')
#             ]
# }    

In [10]:
def change_name_byrow(row):
    """ Change the name of the experiment as the naming in the ini files have spaces
    for each row
    
    Parameters
    ----------
    row : single row from a given Pandas DataFrame
    Returns
    -------
    row : single row from a given Pandas DataFrame

    
    """
    # ini naming is selected as the index column
    # load the lookup table for naming
    exp_names = pd.read_csv(op.join(scriptpath, 'data', 'experiment_naming.csv'), index_col=0)
    #get the related name
    old_name = row['name']
    new_name = exp_names.loc[row['name']] 
    #change the name
    print('Changed name %s to %s' %(old_name, new_name))
    return new_name

In [11]:
def change_name(df):
    """ Change the name of the experiment as the naming in the ini files have spaces
    
    Parameters
    ----------
    df : pd.DataFrame
        A Pandas DataFrame generated from a psychophysics log.
        
    Returns
    -------
    df : pd.DataFrame
        A cleaned up Pandas DataFrame.
    
    """
    # ini naming is selected as the index column
    # load the lookup table for naming
    exp_names = pd.read_csv(op.join(scriptpath, 'data', 'experiment_naming.csv'), index_col=0)
    #get the related name
    new_name = exp_names.loc[df.loc[0, 'name']] 
    #change the name
    df.loc[0,'name']= new_name[0]
    return df

In [24]:
#helper function for user input data for pts_number_processing
def _add_pts_number(user_input):
    if pd.isnull(user_input):
            logger.info('OK, you are not adding any file, this may cause a problem later.')
            return np.nan
    elif user_input.isdigit():
        user_input = int(user_input)
        #check if it is within meaningful limits
        if user_input >=0 or user_input<=100:
            logger.info('You are adding the integer  %i' %user_input)
            return user_input
        
        else:
            logger.info('Your value does not make sense.')
            logger.info('No number is added')
            return np.nan
    else:
        logger.info('Not a valid type, nothing is added.')
        return np.nan

In [25]:
#fixes the problem with string integer conflict for PTS number
def process_ptsnumber(row):
    
    #first check if this is a multi electrode experiment:
    if row['stim_class'] == 'MultiElectrode':
        val = row['pts_number']
        #if value is a string
        if isinstance(val, str):
            if val.isdigit():
                if change23:
                    if int(val) >2:
                        return 2
                    else:
                        return int(val)
                else:
                        return int(val)
            else: #when val is string 
                #check some common cases 
                if val =='nothing' or val =='-':
                    return 0
                else:
                    user_input = input('There is a string instead of a number,\n'
                                       'Please fix %s\n'
                                       'Description is: %s'%(val, row['pts_description']))
                    inp = _add_pts_number(user_input)
                    print(inp)
                    return inp
        #if value is nan
        elif pd.isnull(val):
            user_input = input('No number is added for file %s'
                               '\n in experiment %s, \n could you fix this problem by looking at description \n %s?'
                               %(row['pts_file'],row['name'], row['pts_description'] ))
            
            inp = _add_pts_number(user_input)
            print(inp)
            return inp
            
            #if value is float
        elif isinstance(val, float) or isinstance(val, int):
            if change23:
                if int(val) >2:
                    return 2
                else:
                    return int(val)
            else:
                return int(val)
        else:
            print('This is weird')

In [15]:
def expand_header(df):
    """Parse first row (header row) and expand into columns
    
    The first row typically contains information about the experiment,
    such as subject ID and experimenter ID.
    
    The first row will be deleted, but the entries from this row will be
    applied to all remaining rows.
    
    Parameters
    ----------
    df : pd.DataFrame
        A Pandas DataFrame generated from a psychophysics log.
        
    Returns
    -------
    df : pd.DataFrame
        A cleaned up Pandas DataFrame.
    
    """
    assert not pd.isnull(df.loc[0, 'subject_id'])
    df['subject_id'] = df.loc[0, 'subject_id']
    df['experimenter_id'] = df.loc[0, 'experimenter_id']
    df['name'] = df.loc[0, 'name']
    
    # Tactile files have additional columns:
    if 'dist2board' in df.columns:
        df['dist2board'] = df.loc[0, 'dist2board']
    if 'dist2screen' in df.columns:
        df['dist2screen'] = df.loc[0, 'dist2screen']
    
    # Remove clutter:
    dropcols = ['pts_cd_id', 'pts_cd_version', 'pts_cd_fingerprint',
                'PTS_generatorfunction', 'Trial No.', 'milliseconds',
                'relative_time', 'stimplay', 'status', 'vpu_id']
    for col in dropcols:
        if col in df.columns:
            df.drop(columns=col, inplace=True)
    return df

In [16]:
def get_electrode_name(e_name, leading_zeros=True):
    """Returns electrode name either as e.g. A8 or A08"""
    if leading_zeros:
        return '%s%02d' % (e_name[0], int(e_name[1:]))
    return '%s%d' % (e_name[0], int(e_name[1:]))

In [17]:
def add_img_col(df, drawingpath, verbose=True):
    """Matches up trials with images and adds filename as column
    
    Files must be saved with a leading zero for trial number (e.g.,
    "01_*.png", not "1_*.png"), and live in `drawingpath`.
    
    The function assumes that invalid trials do not affect the
    trial numbering.
    
    Parameters
    ----------
    df : pd.DataFrame
        A Pandas DataFrame generated from a psychophysics log
    drawingpath : str
        Directory in which to look for images
    verbose : bool
        If True, prints status messages
    """
    import glob
    
    
    #First check if the file exists in the drawing path
    # file name to check 
    filename = op.join(fulldrawingpath,
                       os.path.basename(csvfile)).replace(".csv", "_clean.csv")
    if op.exists(filename):
        #load in the old dataframe
        #old_df = argus_shapes.load_data(filename)
        old_df = pd.read_csv(filename)
        #find indices where the data is missing for image column
        indices = old_df['PTS_FILE'].index[old_df['PTS_FILE'].isnull()]
        #if any(indices):
            
    # Drop invalid trials
    df = df[df.PTS_VALID == 1].reset_index()   
    img_col = []
    for idx, row in df.iterrows():
        # png file given by trial no. and electrode 
        files = []
        if 'PTS_ELECTRODE' in df.columns:
            #single electrode
            if pd.isnull(row['PTS_ELECTRODE']):
                continue
            if np.isclose(float(row['PTS_AMP']), 0) and row['name'] != 'Drawings (tactile)':
                # This is a catch trial, look for the word CATCH:
                filestr = filestr = '%02d_*CATCH*.png' % (idx + 1)
                files += glob.glob(os.path.join(drawingpath, filestr),recursive=True)
            elif row['name'] != 'Drawings (tactile)':
                # Not a catch trial
                for leading_zeros in [True, False]:
                    e_name = get_electrode_name(row['PTS_ELECTRODE'], leading_zeros=leading_zeros)
                    # Make sure to save trial number with leading zero:
                    filestr = '%02d_*_%s_*.png' % (idx + 1, e_name)
                    files += glob.glob(os.path.join(drawingpath, filestr), recursive=True)
            else:
                filestr = '%02d_*_%s_*.png' % (idx + 1, row['PTS_ELECTRODE'])
                files = glob.glob(os.path.join(drawingpath, filestr),recursive=True)
        else:
            #double electrodes
            if pd.isnull(row['PTS_ELECTRODE1']) or pd.isnull(row['PTS_ELECTRODE2']):
                continue
            if np.isclose(float(row['PTS_AMP1']), 0) or  np.isclose(float(row['PTS_AMP2']), 0):
                # This is a catch trial, look for the word CATCH:
                filestr = filestr = '%02d_*CATCH*.png' % (idx + 1)
                files += glob.glob(os.path.join(drawingpath,'**', filestr),recursive=True)
            else:
                # This is not a catch trial!!
                for leading_zeros in [True, False]:
                    e_name1 = get_electrode_name(row['PTS_ELECTRODE1'], leading_zeros=leading_zeros)
                    e_name2 = get_electrode_name(row['PTS_ELECTRODE2'], leading_zeros=leading_zeros)
                    # Try both E1E2 and E2E1:
                    filestr = '%02d_*_%s%s_*.png' % (idx + 1, e_name1, e_name2)
                    files += glob.glob(os.path.join(drawingpath, '**', filestr),recursive=True)
                    filestr = '%02d_*_%s%s_*.png' % (idx + 1, e_name2, e_name1)
                    files += glob.glob(os.path.join(drawingpath, '**', filestr), recursive=True)
        files = np.unique(files)
        if len(files) == 1:
            img_col.append(files[0])
            logger.info('  - Trial %d: File added for %s: "%s" \n' % (idx, filestr, files[0]))
        else:
            if len(files) == 0:
                if verbose:
                    logger.info('  - Trial %d: No file found that matches "%s".' % (idx, filestr))
                    #ask if user knows where the file is
                    user_input = input('If you can find the file, please add it.')
                    logger.info('If you can find the file, please add it. \n %s' %user_input)

                    if filestr == user_input:
                        img_col.append(file_chosen)
                        logger.info('  - Trial %d: You added %s \n' %(idx, os.path.basename(user_input)))
                    else:
                        img_col.append(np.nan)
                        logger.info('  - Trial %d: Nothing added. \n')
                        
            else:
                if verbose:
                    logger.info('  - Trial %d: Found multiple files:' % idx)
                    for f in files:
                        
                        logger.info('    - %s' % os.path.basename(f))
                    filename = input("Please type the name of the file if you want to select one of them: ")
                    logger.info('Please type the name of the file if you want to select one of them:  %s' %filename)

                    chosen_files  = [s for s in files if filename in s]
                    if bool(chosen_files):
                        for chosen_file in chosen_files:    
                            img_col.append(chosen_file)
                            logger.info('  - Trial %d: You added %s \n' %(idx, os.path.basename(chosen_file)))
                    else:
                        img_col.append(np.nan)
                        logger.info('  - Trial %d: You did not enter any valid filename, not adding any \n' %idx)
    df['PTS_FILE'] = pd.Series(img_col, index=df.index)
    return df

In [18]:
def clean_up(df, sessionfolder):
    """Adds some columns """
    df['stim_class'] = np.nan
    df['date'] = np.nan
    for idx, row in df.iterrows():
        if 'PTS_ELECTRODE1' in row and 'PTS_ELECTRODE2' in row:
            df.loc[idx, 'stim_class'] = 'MultiElectrode'
        else:
            df.loc[idx, 'stim_class'] = 'SingleElectrode'
        if 'time' in df.columns and not pd.isnull(row['time']):
            datetime = row['time'].split(" ")
            df.loc[idx, 'date'] = datetime[0]
            df.loc[idx, 'time'] = datetime[1]
            
    if 'stimulus' in df.columns:
        df.drop(columns='stimulus', inplace=True)
    return df

In [21]:
subjects_sessions

[('12-005', ['2019_05_29', '2019_10_09']),
 ('12-104', ['2019_05_30', '2019_10_08']),
 ('13-101', ['2019_05_31', '2019_10_07'])]

In [ ]:

if run_trials2drawings:
    for subject, sessions in subjects_sessions:
        logger.info('\n%s' % subject)
        for session in sessions:
            logger.info('\n%s' % session)
            for experiment, subfolders in six.iteritems(drawing_folder):
                logger.info('\n%s' % experiment)
                transferfolder = os.path.join(datapath, subject, session,
                                              'transfer_data')
                csvfiles = argus_scripts.find_csv(transferfolder, experiment)
                if not csvfiles:
                    logger.info('- No csv %s found in %s, skip' % (experiment, transferfolder))
                    continue
                
                # 1 or more files found:
                for csvfile in csvfiles:
                    df = pd.read_csv(csvfile)
                    df = change_name(df)
                    df = expand_header(df)

                    for drawingpath in subfolders:
                        logger.info('Reading %s' % csvfile)
                        fulldrawingpath = os.path.join(datapath, subject, session,
                                                       drawingpath)
                        if not op.exists(fulldrawingpath):
                            logger.info('- No folder %s, skip' % fulldrawingpath)
                            continue
                        newcsvfile = op.join(fulldrawingpath,
                                                  os.path.basename(csvfile)).replace(".csv", "_clean.csv")
                        #if op.exists(newcsvfile):
                            #check if all images are there
                        
                        logger.info('- Parsing %s' % fulldrawingpath)
                        dff = df.copy(deep=True)
                        if run_addimgcol:
                            dff = add_img_col(dff, fulldrawingpath)
                        dff = clean_up(dff, os.path.join(datapath, subject, session))
                        dff.to_csv(newcsvfile, index=False)
                        logger.info('  - Saved to %s' % newcsvfile)
                    

In [19]:
drawing_main = 'drawings_filled'
drawing_subfolder = [ 
                    'drawings_double'
]


In [26]:
## 3. Generate a main dataframe
if run_collatedata:
    #generate the collated the dataframe
    subjects = 'S_'
    data_list = []
    collated = pd.DataFrame()
    for subject, sessions in subjects_sessions:
        #print(subject)
        for session in sessions:
            #print(session)
            for  drawingpath in drawing_subfolder:
                        #print(drawingpath)
                        fulldatapath = os.path.join(datapath, subject, session, drawing_main, drawingpath)
                        #print(fulldatapath)
                        if os.path.isdir(fulldatapath):
                            print('This is a directory')
                            csvname = fulldatapath + '/*_clean.csv'
                            #print(csvname)
                            #get the fileslist
                            csvfile = glob.glob(csvname)
                           # print(csvfile)
                            for files in csvfile:
                                print(files)
                                df = pd.read_csv(files)
                                data_list.append(df)
                               #print(len(data_list))
        #for naming purposes
        subjects += subject + '_'
    collated = pd.concat(data_list)                              
    #change name of experiments if it hasn't been changed
    collated['name']=collated.apply(change_name_byrow, axis=1)
    collated.columns = map(str.lower, collated.columns)
    collated['pts_number_processed'] = collated.apply(process_ptsnumber, axis =1)
    #save colladted as csv in argus_scripts data folder
    collated_file = op.join(scriptpath, 'data',  subjects + '_collated_both_sessions.csv')
    collated.to_csv(collated_file, index=False)

   

This is a directory
/mnt/viscog/FineLab/Data/12-005/2019_05_29/drawings_filled/drawings_double/psychophysicslog_20190529_200230_078_1_clean.csv
This is a directory
/mnt/viscog/FineLab/Data/12-005/2019_10_09/drawings_filled/drawings_double/psychophysicslog_20191009_160711_000_1_clean.csv
This is a directory
/mnt/viscog/FineLab/Data/12-104/2019_05_30/drawings_filled/drawings_double/psychophysicslog_20190530_193914_453_1_clean.csv
This is a directory
/mnt/viscog/FineLab/Data/12-104/2019_10_08/drawings_filled/drawings_double/psychophysicslog_20191008_190540_968_1_clean.csv
/mnt/viscog/FineLab/Data/12-104/2019_10_08/drawings_filled/drawings_double/psychophysicslog_20191008_201432_968_1_clean.csv
/mnt/viscog/FineLab/Data/12-104/2019_10_08/drawings_filled/drawings_double/psychophysicslog_20191008_195151_015_1_clean.csv
/mnt/viscog/FineLab/Data/12-104/2019_10_08/drawings_filled/drawings_double/psychophysicslog_20191008_205052_984_1_clean.csv
/mnt/viscog/FineLab/Data/12-104/2019_10_08/drawings_

Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: dr

Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: dr

Changed name drawing_double_Thr_1 to exp_name    drawing_double_Thr_1
Name: drawing_double_Thr_1, dtype: object
Changed name drawing_double_Thr_1 to exp_name    drawing_double_Thr_1
Name: drawing_double_Thr_1, dtype: object
Changed name drawing_double_Thr_1 to exp_name    drawing_double_Thr_1
Name: drawing_double_Thr_1, dtype: object
Changed name drawing_double_Thr_1 to exp_name    drawing_double_Thr_1
Name: drawing_double_Thr_1, dtype: object
Changed name drawing_double_Thr_1 to exp_name    drawing_double_Thr_1
Name: drawing_double_Thr_1, dtype: object
Changed name drawing_double_Thr_1 to exp_name    drawing_double_Thr_1
Name: drawing_double_Thr_1, dtype: object
Changed name drawing_double_Thr_1 to exp_name    drawing_double_Thr_1
Name: drawing_double_Thr_1, dtype: object
Changed name drawing_double_Thr_1 to exp_name    drawing_double_Thr_1
Name: drawing_double_Thr_1, dtype: object
Changed name drawing_double_2Thr_1 to exp_name    drawing_double_2Thr_1
Name: drawing_double_2Thr_1, dty

Changed name drawing_double_Thr_1 to exp_name    drawing_double_Thr_1
Name: drawing_double_Thr_1, dtype: object
Changed name drawing_double_Thr_1 to exp_name    drawing_double_Thr_1
Name: drawing_double_Thr_1, dtype: object
Changed name drawing_double_Thr_1 to exp_name    drawing_double_Thr_1
Name: drawing_double_Thr_1, dtype: object
Changed name drawing_double_Thr_1 to exp_name    drawing_double_Thr_1
Name: drawing_double_Thr_1, dtype: object
Changed name drawing_double_Thr_1 to exp_name    drawing_double_Thr_1
Name: drawing_double_Thr_1, dtype: object
Changed name drawing_double_Thr_1 to exp_name    drawing_double_Thr_1
Name: drawing_double_Thr_1, dtype: object
Changed name drawing_double_2Thr_0 to exp_name    drawing_double_2Thr_0
Name: drawing_double_2Thr_0, dtype: object
Changed name drawing_double_2Thr_0 to exp_name    drawing_double_2Thr_0
Name: drawing_double_2Thr_0, dtype: object
Changed name drawing_double_2Thr_0 to exp_name    drawing_double_2Thr_0
Name: drawing_double_2Thr_

...

0
No number is added for file /mnt/viscog/FineLab/Data/13-101/2019_10_07/drawings_filled/drawings_double/31_13-101_CATCH_2019-10-29_14-20-13.png
 in experiment drawing_double_2Thr_1, 
 could you fix this problem by looking at description 
 nan?0


You are adding the integer  0


0
No number is added for file /mnt/viscog/FineLab/Data/13-101/2019_10_07/drawings_filled/drawings_double/33_13-101_CATCH_2019-10-29_14-21-25.png
 in experiment drawing_double_2Thr_1, 
 could you fix this problem by looking at description 
 nan?0


You are adding the integer  0


0


## 4. Fill in images

In [ ]:
def fill_shape(fname):
    if not fname:
            print('%s is null'  %fname)
            return None
    else:
        image = skio.imread(fname, as_gray=False)
        image = sk.img_as_float(image)
        r = image[:,:,0]
        g = image[:,:,1]
        b = image[:,:,2]
        # find all red pixels:
        reds = (r > 0.5) & (g < 0.5)  & (b < 0.5)
        filled = ndi.binary_fill_holes(reds)
        filled = sk.img_as_float(filled)
    
        print('Filled image %s \n'%fname)
        return filled

In [ ]:
# idk how to elegantly seperate joined paths in drawing_folder dict therefore I use this
drawing_main = 'drawings_clean'
drawing_subfolder = [ 
                    'drawings_single',
                    'drawings_double']


if run_clean2filled:
    for subject, sessions in subjects_sessions:
        for session in sessions:
            for  subfolder in drawing_subfolder:
                #get the directory
                fulldrawingpath = os.path.join(datapath, subject, session, drawing_main, subfolder)
                if os.path.isdir(fulldrawingpath):
                    csvname = fulldrawingpath + '/*_clean.csv'
                    #get the fileslist
                    csvfile = glob.glob(csvname)
                    print(csvfile)
                    for files in csvfile:
                        df = pd.read_csv(files)
                        #process the image
                        for idx,row in df.iterrows():
                            filename = row['PTS_FILE']
                            if pd.isnull(filename):
                                continue
                            tmp = fill_shape(filename)
                            #save into filled
                            filledpath = os.path.dirname(filename.replace('clean', 'filled'))
                            if not os.path.exists(filledpath):
                                os.makedirs(filledpath)
                                print(' Created new directory %s' %filledpath)
                            img_name = os.path.join(filledpath , op.basename(filename))
                            df.loc[idx,'PTS_FILE'] = img_name
                            print('saved image %s' %img_name)
                            skio.imsave(img_name, tmp)

                    newcsvfile = os.path.join(filledpath, csvfile[0].split('/')[-1])
                    df.to_csv(newcsvfile, index=False)
                    print('  - Saved to %s' % newcsvfile)